In [3]:
import os
import subprocess
import time
import sys
import threading
import socket
from urllib.request import Request, urlopen
import re

def install_packages():
    subprocess.check_call([sys.executable, "-m", "pip", "install", "streamlit", "pyngrok", "pyjwt", "watchdog"])
print("Installing required packages...")
install_packages()

from pyngrok import ngrok

os.makedirs(".streamlit", exist_ok=True)
config_toml = """
[theme]
base="dark"
primaryColor="#4F8BF9"
backgroundColor="#0E1117"
secondaryBackgroundColor="#262730"
textColor="#FAFAFA"
font="sans serif"
[server]
headless = true
"""
with open(".streamlit/config.toml", "w") as f:
    f.write(config_toml)
print("Applied Dark Theme configuration.")

Installing required packages...
Applied Dark Theme configuration.


In [5]:
%%writefile app.py
import streamlit as st
import jwt
import datetime
import time
import re
import sqlite3
import hashlib


SECRET_KEY = "super_secret_key_for_demo"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30


def get_db():
    conn = sqlite3.connect("users.db", check_same_thread=False)
    conn.row_factory = sqlite3.Row
    return conn

conn = get_db()
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT UNIQUE NOT NULL,
    email TEXT UNIQUE NOT NULL,
    password TEXT NOT NULL,
    security_question TEXT NOT NULL,
    security_answer TEXT NOT NULL
)
""")
conn.commit()


def create_access_token(data: dict):
    to_encode = data.copy()
    expire = datetime.datetime.utcnow() + datetime.timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    to_encode.update({"exp": expire})
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)

def verify_token(token: str):
    try:
        return jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
    except:
        return None


def clean(v):
    return v.strip()

def is_valid_email(email):
    return re.match(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$', email)

def is_valid_password(password):
    return len(password) >= 8 and password.isalnum()

def hash_password(p):
    return hashlib.sha256(p.encode()).hexdigest()


if 'jwt_token' not in st.session_state:
    st.session_state['jwt_token'] = None
if 'page' not in st.session_state:
    st.session_state['page'] = 'login'


st.set_page_config(page_title="Infosys SpringBoard Intern", page_icon="🤖", layout="wide")

st.markdown("""
<style>
.stApp { background-color: #0E1117; }
h1 { text-align: center; color: #4F8BF9; }
h3 { text-align: center; color: #FAFAFA; }
.stButton>button {
    width: 100%; border-radius: 8px; height: 3em;
    background-color: #4F8BF9; color: white; font-weight: bold;
}
.stButton>button:hover { background-color: #3b6ccf; }
.user-msg {
    text-align:right; background:#262730; color:white;
    padding:10px; border-radius:10px; margin:5px; float:right; clear:both;
}
.bot-msg {
    text-align:left; background:#4F8BF9; color:white;
    padding:10px; border-radius:10px; margin:5px; float:left; clear:both;
}
</style>
""", unsafe_allow_html=True)


def login_page():
    col1, col2, col3 = st.columns([1,2,1])
    with col2:
        st.title("Infosys SpringBoard Intern")
        st.markdown("<h3>Please sign in to continue</h3>", unsafe_allow_html=True)

        with st.form("login_form"):
            email = clean(st.text_input("Email Address"))
            password = st.text_input("Password", type="password")
            submit = st.form_submit_button("Sign In")

            if submit:
                cursor.execute("SELECT * FROM users WHERE email=?", (email,))
                user = cursor.fetchone()

                if user and user["password"] == hash_password(password):
                    token = create_access_token({"sub": email, "username": user["username"]})
                    st.session_state['jwt_token'] = token
                    st.success("Login successful!")
                    time.sleep(0.5)
                    st.rerun()
                else:
                    st.error("Invalid email or password")

        if st.button("Forgot Password?"):
            st.session_state.page = "forgot"
            st.rerun()

        if st.button("Create an Account"):
            st.session_state.page = "signup"
            st.rerun()


def signup_page():
    col1, col2, col3 = st.columns([1,2,1])
    with col2:
        st.title("Create Account")

        with st.form("signup_form"):
            username = clean(st.text_input("Username"))
            email = clean(st.text_input("Email Address"))
            password = st.text_input("Password", type="password")
            confirm = st.text_input("Confirm Password", type="password")
            question = st.selectbox("Security Question", [
                "What is your pet name?",
                "What is your mother's maiden name?",
                "What is your favorite teacher?"
            ])
            answer = clean(st.text_input("Security Answer"))
            submit = st.form_submit_button("Sign Up")

            if submit:
                if not all([username,email,password,confirm,answer]):
                    st.error("All fields are mandatory")
                elif not is_valid_email(email):
                    st.error("Invalid email format")
                elif not is_valid_password(password):
                    st.error("Weak password")
                elif password != confirm:
                    st.error("Passwords do not match")
                else:
                    try:
                        cursor.execute(
                            "INSERT INTO users VALUES(NULL,?,?,?,?,?)",
                            (username,email,hash_password(password),question,answer.lower())
                        )
                        conn.commit()
                        st.session_state.jwt_token = create_access_token({"sub":email,"username":username})
                        st.success("Account created successfully!")
                        st.rerun()
                    except:
                        st.error("Email or username already exists")

        if st.button("Back to Login"):
            st.session_state.page = "login"
            st.rerun()


def forgot_page():
    col1, col2, col3 = st.columns([1, 2, 1])
    with col2:
        st.title("Forgot Password")

        email = clean(st.text_input("Registered Email"))


        if st.button("Verify"):
            if not email:
                st.error("Email cannot be empty")
                return

            cursor.execute("SELECT * FROM users WHERE email=?", (email,))
            user = cursor.fetchone()

            if not user:
                st.error("Email not registered")
                return


            st.session_state.reset = email
            st.session_state.q = user["security_question"]


        if "q" in st.session_state:
            st.info(st.session_state.q)

            ans = clean(st.text_input("Answer"))  # Free text allowed
            newp = clean(st.text_input("New Password", type="password"))

            if st.button("Reset Password"):

                if "reset" not in st.session_state:
                    st.error("Please verify email first")
                    return

                if not ans or not newp:
                    st.error("All fields are required")
                    return


                if not is_valid_password(newp):
                    st.error("Password must be alphanumeric and at least 8 characters")
                    return

                cursor.execute(
                    "SELECT * FROM users WHERE email=? AND security_answer=?",
                    (st.session_state.reset, ans.lower())
                )

                if cursor.fetchone():
                    cursor.execute(
                        "UPDATE users SET password=? WHERE email=?",
                        (hash_password(newp), st.session_state.reset)
                    )
                    conn.commit()


                    del st.session_state.reset
                    del st.session_state.q

                    st.success("Password updated successfully")
                    st.session_state.page = "login"
                    st.rerun()
                else:
                    st.error("Incorrect security answer")


def dashboard_page():
    token = st.session_state.get('jwt_token')
    payload = verify_token(token)

    if not payload:
        st.session_state['jwt_token'] = None
        st.warning("Session expired or invalid. Please login again.")
        time.sleep(1)
        st.rerun()
        return
    username = payload.get("username", "User")

    with st.sidebar:
        st.title("🤖 LLM")
        st.markdown("---")
        if st.button("➕ New Chat", use_container_width=True):
             st.info("Started new chat!")

        st.markdown("### History")
        st.markdown("- Project analysis")
        st.markdown("- NLP")
        st.markdown("---")
        st.markdown("### Settings")
        if st.button("Logout", use_container_width=True):
            st.session_state['jwt_token'] = None
            st.rerun()

    st.title(f"Welcome, {username}!")
    st.markdown("### How can I help you today?")


    chat_placeholder = st.empty()

    with chat_placeholder.container():
        st.markdown('<div class="bot-msg">Hello! I am LLM. Ask me anything about LLM!</div>', unsafe_allow_html=True)



    with st.form(key='chat_form', clear_on_submit=True):
        col1, col2 = st.columns([6, 1])
        with col1:
            user_input = st.text_input("Message LLM...", placeholder="Ask me anything about LLM...", label_visibility="collapsed")
        with col2:
            submit_button = st.form_submit_button("Send")

        if submit_button and user_input:

             st.markdown(f'<div class="user-msg">{user_input}</div>', unsafe_allow_html=True)
             st.markdown('<div class="bot-msg">I am a demo bot. I received your message!</div>', unsafe_allow_html=True)


if st.session_state.jwt_token:
    dashboard_page()
else:
    if st.session_state.page == "signup":
        signup_page()
    elif st.session_state.page == "forgot":
        forgot_page()
    else:
        login_page()

Overwriting app.py


In [ ]:

# --- Wait for Streamlit to Start ---
def wait_for_streamlit(port=8501, timeout=30):
    start_time = time.time()
    while time.time() - start_time < timeout:
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(1)
            result = sock.connect_ex(('localhost', port))
            if result == 0:
                sock.close()
                return True
            sock.close()
        except Exception:
            pass
        time.sleep(1)
    return False
# --- Ngrok Setup ---
print("\nTo access the app, you need an Ngrok Authtoken.")
print("Get it from: https://dashboard.ngrok.com/get-started/your-authtoken")
authtoken = input("Enter your Ngrok Authtoken: ").strip()
if authtoken:
    ngrok.set_auth_token(authtoken)

    # Kill any existing ngrok process
    os.system("pkill ngrok")
    os.system("pkill streamlit")

    # Run Streamlit in the background FIRST
    print("Starting Streamlit...")
    # Using Subprocess.Popen to run in background
    # Redirecting output to /dev/null to keep cell clean
    process = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501", "--server.address", "localhost"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Wait for it to be ready
    if wait_for_streamlit():
        print("Streamlit is active! Connecting Ngrok...")
        # Open a tunnel to the streamlit port 8501
        try:
            public_url = ngrok.connect(8501).public_url
            print(f"\n🚀 Streamlit App is running!")
            print(f"👉 Public URL: {public_url}")
            print("\n(Click the URL above to open the app)")

            # Keep main thread alive
            try:
                # Keep checking if process is alive
                while process.poll() is None:
                    time.sleep(1)
            except KeyboardInterrupt:
                print("Stopping...")
                ngrok.disconnect(public_url)
                process.terminate()
        except Exception as e:
            print(f"Ngrok connection failed: {e}")
            process.terminate()
    else:
        print("Error: Streamlit failed to start in time.")
        process.terminate()
else:
    print("Ngrok Authtoken is required to expose the app publicly.")


To access the app, you need an Ngrok Authtoken.
Get it from: https://dashboard.ngrok.com/get-started/your-authtoken
Enter your Ngrok Authtoken: 3AHkay8NPWfBPfFwSClyn22HZro_7TFrsVDRQXtgC2BFceyF9
Starting Streamlit...
Streamlit is active! Connecting Ngrok...

🚀 Streamlit App is running!
👉 Public URL: https://patty-unpolled-nicolle.ngrok-free.dev

(Click the URL above to open the app)
